# Verabeitung der Artikel und Fragen

##### Quellen:

```
Pandas. (2025, 7. Juli). Pandas Documentation. Abgerufen am 11. Juli 2025, von https://pandas.pydata.org/docs/


```

In [1]:
import pandas as pd
import os 
import re

#### Excel mit den Artikel einlesen und säubern

In [2]:

df = pd.read_excel("artikel_tagesschaauu.xlsx", header=3)


In [3]:
spalten_ignorieren = ['Unnamed: 0', 'Unnamed: 1']
df = df.drop(columns=[s for s in spalten_ignorieren if s in df.columns])
df

,Link,Titel,Text,Datum,Spalte1
0,https://www.tagesschau.de/ausland/uswahl/usa-b...,Biden schafft schnell noch politische Fakten,Am 20. Januar übergibt US-Präsident Biden sein...,19.11.2024,NaN
1,https://www.tagesschau.de/ausland/uswahl/trump...,Was Trumps Personal für die Außenpolitik bedeutet,"Verteidigungsminister, UN-Vertreterin oder Bot...",2024-11-16 00:00:00,NaN
2,https://www.tagesschau.de/ausland/asien/china-...,Nutzt China die Trump-Wahl für eine Charme-Off...,Ob G20 oder APEC - China hatte sich von intern...,2024-11-15 00:00:00,NaN
3,https://www.tagesschau.de/ausland/europa/trump...,Nützt Trump den Nationalisten auf dem Balkan?,Die USA sind auch für die Sicherheit des Balka...,2024-11-17 00:00:00,NaN
4,https://www.tagesschau.de/ausland/uswahl/taiwa...,Würde Trump Taiwan verteidigen?,Der von China bedrohte Inselstaat Taiwan wird ...,2024-11-10 00:00:00,NaN
...,...,...,...,...,...
90,https://www.tagesschau.de/ausland/amerika/usa-...,Trump für Angriff auf Irans Atomanlagen,Im US-Wahlkampf hat sich der republikanische P...,2024-10-05 00:00:00,NaN
91,https://www.tagesschau.de/ausland/selenskyj-ha...,Versprechen an Selenskyj - Kritik an Trump,Im Falle ihres Wahlsieges will US-Präsidentsch...,2024-09-27 00:00:00,NaN
92,https://www.tagesschau.de/ausland/amerika/desi...,USA werfen Russland Einmischung in Präsidenten...,"Die US-Regierung beschuldigt Russland, sich in...",2024-09-04 00:00:00,NaN
93,https://www.tagesschau.de/ausland/uswahl/fake-...,"Taylor, Trump und Russland - Fakes als Waffe",Der US-Wahlkampf geht in die heiße Endphase - ...,2024-09-15 00:00:00,NaN


#### Zeitspanne der Artikel herausfinden

In [4]:
df["Datum"] = pd.to_datetime(df["Datum"])


df["Datum"].describe()


C:\Users\Privat\AppData\Local\Temp\ipykernel_9816\1180972687.py:1: UserWarning: Parsing dates in  %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Datum"] = pd.to_datetime(df["Datum"])


count                               95
mean     2024-10-26 11:06:56.842105344
min                2024-07-23 00:00:00
25%                2024-10-27 12:00:00
50%                2024-11-06 00:00:00
75%                2024-11-10 00:00:00
max                2024-11-24 00:00:00
Name: Datum, dtype: object

#### Artikeltext extrahieren und in seperaten txt Dateien speichern

In [5]:
def speichere_artikel_als_txt(df, zielverzeichnis):
    """
    Extrahiert den Text der Artikel aus der Excel und speichert 
    ihn in als jeweils sperate txt Dateien.

    """

    # Texte speichern
    for i, text in enumerate(df["Text"], start=1):
        dateiname = f"artikel_{i}.txt"
        dateipfad = os.path.join(zielverzeichnis, dateiname)
        with open(dateipfad, 'w', encoding='utf-8') as f:
            f.write(str(text))

    print(f"{len(df)} Artikel wurden in {zielverzeichnis} gespeichert.")

In [ ]:
speichere_artikel_als_txt(df,"originalartikel")

95 Artikel wurden in test gespeichert.


#### Fragen und Musterlösungen parsen und diese mit anderen Merkmalen in einem Df speichern

**Das erstellte Df ist Evaluationsgrundlage, es wird mit den Antworten der Systeme erweitert**

In [11]:

# Pfad zu dem Ordner mit den Fragen und Musterlösungen
ordnerpfad = "fragen_und_loesungen"

def parse_fragen_muster(file_path, artikel_id):
    """
    Parst Datei mit erstellten Fragen und
    Musterlösungen des LLMs zu den Manipulierten Artikeln 
    """

    # Kompletten Inhalt der Datei einlesen
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    try:
        # Text zwischen Fragen und Musterlösungen
        fragen_block = re.search(r"\*\*Fragen:\*\*(.*?)\*\*Musterlösungen:\*\*", content, re.S).group(1)
        # Text nach Musterlösungen
        muster_block = re.search(r"\*\*Musterlösungen:\*\*(.*)", content, re.S).group(1)
    except Exception as e:
        # Falls ein Abschnitt nicht gefunden wird
        print(f"Fehler beim Parsen von Datei: {file_path, e}")
        return None
    
    # Fragen extrahieren im format 1. Frage... 2. Frage ... 3. Frage ...
    fragen = re.findall(r"\d+\.\s(.*?)\n", fragen_block)

    # Musterlösungen extrahieren im selben Format extrahieren
    muster = re.findall(r"\d+\.\s(.*?)(?:\n|$)", muster_block)


    # Prüfen ob Anzahl der Fragen mit Anzahl der Musterlösungen übereinstimmt
    if len(fragen) != len(muster):
        print(f"Längenproblem in Datei: {file_path} ({len(fragen)} Fragen, {len(muster)} Musterlösungen)")
        return None
    
    # Kategorien für Fake News Arten
    fake_news_types = ["Fake Fact", "Framing", "Exaggeration"]


    # Df mit den Extrahierten Infos erzeugen
    df = pd.DataFrame({
        "artikel_id": artikel_id,
        "frage_nr": list(range(1, len(fragen)+1)),
        "frage": fragen,
        "musterloesung": muster,
        "fake_news_art": fake_news_types
    })

    return df

# Sammeln aller dfs
alle_dfs = []

# Über die Fragen u. Müsterlösungen in dem ordner fragen_und_loesungen itereien 
for filename in os.listdir(ordnerpfad):
    # prüfen ob datei auf "_fragen.txt endet"
    if filename.endswith("_fragen.txt"):
        # Artikel-ID aus dem Dateinamen extrahieren
        artikel_id_match = re.search(r"artikel_(\d+)_fragen.txt", filename)
        # id in int umwandeln
        artikel_id = int(artikel_id_match.group(1))
        # Pfad zur Datei erzeugen
        file_path = os.path.join(ordnerpfad, filename)
        # Datei parsen
        df = parse_fragen_muster(file_path, artikel_id)
        # zur liste hinzufügen
        if df is not None:
            alle_dfs.append(df)
            
# Alle dfs zu einem zusammenführen
df_alle_fragen = pd.concat(alle_dfs, ignore_index=True)



In [12]:
df_alle_fragen

,artikel_id,frage_nr,frage,musterloesung,fake_news_art
0,10,1,Wer wurde von Trump für das Amt des Justizmini...,Der erzkonservative Abgeordnete Matt Gaetz wur...,Fake Fact
1,10,2,Wie wird Gaetz in Bezug auf seine Verbindung z...,Gaetz gilt als Trump-Loyalist und einer der ve...,Framing
2,10,3,Wie wurde die Nominierung von Gaetz zum Justiz...,Die Nominierung von Gaetz zum Justizminister w...,Exaggeration
3,11,1,"Wer wurde von Donald Trump als ""Grenzzar"" erna...","Tom Homan wurde von Donald Trump als ""Grenzzar...",Fake Fact
4,11,2,"Wie hat Trump seine Entscheidung für den ""Gren...",Trump begründete seine Entscheidung für Tom Ho...,Framing
...,...,...,...,...,...
280,95,2,Wie hat Meta seinen Schritt gegen die russisch...,"Meta hat den Schritt mit dem Vorwurf der ""ausl...",Framing
281,95,3,Wie ist der aktuelle Status von RT auf der Pla...,Auf der Plattform X (ehemals Twitter) sind die...,Exaggeration
282,9,1,Welche Rolle hat Mike Waltz in Bezug auf den K...,Mike Waltz ist als Verfechter eines harten Kur...,Fake Fact
283,9,2,Wie wurde Mike Waltz' Meinung zu China im Orig...,Im Originalartikel wurde Mike Waltz' Haltung a...,Framing


## df als csv abspeichern, diese ist Grundlage für die Einspeisung in die RAG-Systeme und die spätere Evaluation

In [21]:
df_alle_fragen.to_csv("alle_fragen_u_loesungen.csv",index=False, encoding="utf-8")

In [22]:
df_alle_fragen

,artikel_id,frage_nr,frage,musterloesung,fake_news_art
0,10,1,Wer wurde von Trump für das Amt des Justizmini...,Der erzkonservative Abgeordnete Matt Gaetz wur...,Fake Fact
1,10,2,Wie wird Gaetz in Bezug auf seine Verbindung z...,Gaetz gilt als Trump-Loyalist und einer der ve...,Framing
2,10,3,Wie wurde die Nominierung von Gaetz zum Justiz...,Die Nominierung von Gaetz zum Justizminister w...,Exaggeration
3,11,1,"Wer wurde von Donald Trump als ""Grenzzar"" erna...","Tom Homan wurde von Donald Trump als ""Grenzzar...",Fake Fact
4,11,2,"Wie hat Trump seine Entscheidung für den ""Gren...",Trump begründete seine Entscheidung für Tom Ho...,Framing
...,...,...,...,...,...
280,95,2,Wie hat Meta seinen Schritt gegen die russisch...,"Meta hat den Schritt mit dem Vorwurf der ""ausl...",Framing
281,95,3,Wie ist der aktuelle Status von RT auf der Pla...,Auf der Plattform X (ehemals Twitter) sind die...,Exaggeration
282,9,1,Welche Rolle hat Mike Waltz in Bezug auf den K...,Mike Waltz ist als Verfechter eines harten Kur...,Fake Fact
283,9,2,Wie wurde Mike Waltz' Meinung zu China im Orig...,Im Originalartikel wurde Mike Waltz' Haltung a...,Framing


## Fake News markierungen Löschen

#### Da die Markierungen für die Fake news nicht in dem Kontext der RAG-Systeme vorhanden sein dürfen, müssen diese Entfernt werden

In [ ]:
import os
import re

# Pfad zum Ordner mit den manipulierten Artikel
eingabe_ordner = "manipulierte_artikel"

# Späterer Ordnerpfad für die manipulierten Artikel ohne Markierung
ausgabe_ordner = "manipulierte_artikel_ohne_markierung"

def entferne_fake_tags(text):
    # Erste Zeile entfernen
    text = re.sub(r"^.*?\n", "", text, count=1)
    # Fake-News-Tags werden entfernt bspw. [[Fake Fact]] ... [[/Fake Fact]] innerer Text bleibt aber erhalten
    return re.sub(r"\[\[(FAKE FACT|FRAMING|EXAGGERATION)\]\](.*?)\[\[/\1\]\]", r"\2", text, flags=re.DOTALL)

# über alle dateien im ordner iterieren
for datei in os.listdir(eingabe_ordner):
    # Datei muss textdatei sein
    if datei.endswith(".txt"):
        # Vollständiger Pfad zur Eingabedatei
        pfad_eingabe = os.path.join(eingabe_ordner, datei)
        # Vollständiger Pfad zur Ausgabedatei
        pfad_ausgabe = os.path.join(ausgabe_ordner, datei)

        # Dateiinhalt lesen
        with open(pfad_eingabe, "r", encoding="utf-8") as f:
            originaltext = f.read()

        # Tags entfernen
        bereinigter_text = entferne_fake_tags(originaltext)

        # Bereiningten Text in neuer Datei speichern
        with open(pfad_ausgabe, "w", encoding="utf-8") as f:
            f.write(bereinigter_text)

print("Alle Artikel wurden bereinigt und gespeichert")


Alle Artikel wurden bereinigt und gespeichert.
